In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

In [ ]:
import spacy
nlp = spacy.blank('en')
textcat = nlp.create_pipe(
              "textcat",
              config={
                "exclusive_classes": True,
                "architecture": "bow"})

nlp.add_pipe(textcat)

In [ ]:
textcat.add_label('yes')
textcat.add_label('no')

In [ ]:
train_text = data.text.values
train_labels = [{'cats': {'yes' : l== 1, 'no': l==0}} for l in data.target]

In [ ]:
train_data = list(zip(train_text, train_labels))

In [ ]:
from spacy.util import minibatch
import random
random.seed(1) #one random process
spacy.util.fix_random_seed(1) #
optimizer = nlp.begin_training()

In [ ]:
losses = {}
for epoch in range(10):
    random.shuffle(train_data)
    batches = minibatch(train_data, size=8)
    for batch in batches:
        texts, labels = zip(*batch) #unzip
        nlp.update(texts, labels, sgd=optimizer, losses = losses)
    print(losses)

In [ ]:
# from exersize
text = ["This tea cup was full of holes. Do not recommend.","Fire! Caution!!! Seek shelter", "Buy me a barbie doll"]
doc = [nlp.tokenizer(t) for t in text]
textcat = nlp.get_pipe('textcat')
scores, _ = textcat.predict(doc) #????
print(scores)

In [ ]:
predict_l = scores.argmax(axis = 1)
print([textcat.labels[l] for l in predict_l])

In [ ]:
test_d = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')


In [ ]:
test = test_d['text'].values
test[:5]

In [ ]:
tt = test
doc = [nlp.tokenizer(t) for t in tt]
textcat = nlp.get_pipe('textcat')
scores, _ = textcat.predict(doc) 
print(scores)

In [ ]:
predict_l = scores.argmin(axis = 1)
print([textcat.labels[l] for l in predict_l])

In [ ]:
tab_submit = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv', header = 0, names = ['id','target'])
tab_submit.to_csv('submission_Ready.csv', index = False)
tab_submit['target'] = predict_l
tab_submit.head()

In [ ]:
tab_submit.to_csv('submission_Ready.csv', index = False)